# Magics, Imports, and Versions

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import json
from pathlib import Path
import pyedflib
import re
from subprocess import call
import sys
import typing
import urllib

import fastai
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

In [7]:
print(f'__Python VERSION: {sys.version}')

try:
    print(f'__pyTorch VERSION: {torch.__version__}')
    PYTORCH = True
except: 
    print("Pytorch Not Installed")
    PYTORCH = False

try:
    print(f'__fastai VERSION: {fastai.__version__}')
except:
    print("fastai Not Installed")
    
print('__CUDA VERSION')
# call(["nvcc", "--version"]) does not work
! nvcc --version
if PYTORCH:
    print(f'__CUDNN VERSION: {torch.backends.cudnn.version()}')
    print(f'__Number CUDA Devices: {torch.cuda.device_count()}')
    
print(f'__Devices')
try:
    call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
    print(f'Active CUDA Device: GPU {torch.cuda.current_device()}')

    print (f'Available devices: {torch.cuda.device_count()}')
    print (f'Current cuda device {torch.cuda.current_device()}')
except:
    print("No GPUs Found")

__Python VERSION: 3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
__pyTorch VERSION: 1.0.1.post2
__fastai VERSION: 1.0.51
__CUDA VERSION
/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 7402
__Number CUDA Devices: 0
__Devices
No GPUs Found


In [8]:
here = Path(f'{os.getcwd()}')
data_path = here.parent/'data'
raw_path = data_path/'raw'/'v1.5.0/edf'

In [9]:
working_files = []
for (dir_path, dir_names, file_names) in os.walk(raw_path):
    max_eeg = [re.search('t(\d*)',file.split('/')[-1])[1] for file in file_names]
    m = max(max_eeg) if len(max_eeg) >= 1 else 0
    for file in file_names:
        if file.endswith('.edf'):
            working_files.append((dir_path + '/' + file, m))

In [10]:
working_files[:5]

[]

In [5]:
# make sure that they all have the same structure
nums = []
for file, m in working_files:
    nums.append(len(file.split('/')))
collections.Counter(nums)

Counter({14: 5610})

In [58]:
d = collections.defaultdict(list)
for file, m in working_files:
    split = file.split('/')
    data_type = split[8]
    config = split[9].split('_')[-1]
    patient_id = split[11]
    long = split[12].split('_')
    session = long[0].strip('s')
    date = f'{long[1]}/{long[2]}/{long[3].strip(patient_id)}' 
    segment = re.search('t(\d*)',split[13])[1]
    f = pyedflib.EdfReader(file)
    channels = str(f.signals_in_file)
    num_obs = str(f.getNSamples()[0])
    label_file = file[:-3]+'tse'
    with open(label_file,'r') as f:
        labels, durations = [], []
        for line in f.readlines()[2:]:
            labels.append(line.split(' ')[2])
            durations.append(line.split(' ')[1])
    sample_rate = str(num_obs / durations[-1])
    d[patient_id].append({'date':date, 
                          'session':session, 
                          'segment':segment, 
                          'total_segments':m, 
                          'labels':labels,
                          'durations': durations,
                          'channels':channels,
                          'number_obs': num_obs,
                          'sample_rate': sample_rate,
                          'config':config, 
                          'type':data_type, 
                          'loc':file,
                          'label_file': label_file
                         })

In [59]:
d['00004151']

[{'date': '2007/09/28',
  'session': '002',
  'segment': '002',
  'total_segments': '003',
  'labels': ['bckg'],
  'durations': ['256.0000'],
  'channels': '41',
  'number_obs': '64000',
  'config': 'le',
  'type': 'dev_test',
  'loc': '/home/jupyter/tuh_eeg/data/raw/v1.5.0/edf/dev_test/02_tcp_le/041/00004151/s002_2007_09_28/00004151_s002_t002.edf',
  'label_file': '/home/jupyter/tuh_eeg/data/raw/v1.5.0/edf/dev_test/02_tcp_le/041/00004151/s002_2007_09_28/00004151_s002_t002.tse'},
 {'date': '2007/09/28',
  'session': '002',
  'segment': '001',
  'total_segments': '003',
  'labels': ['bckg'],
  'durations': ['490.0000'],
  'channels': '41',
  'number_obs': '122500',
  'config': 'le',
  'type': 'dev_test',
  'loc': '/home/jupyter/tuh_eeg/data/raw/v1.5.0/edf/dev_test/02_tcp_le/041/00004151/s002_2007_09_28/00004151_s002_t001.edf',
  'label_file': '/home/jupyter/tuh_eeg/data/raw/v1.5.0/edf/dev_test/02_tcp_le/041/00004151/s002_2007_09_28/00004151_s002_t001.tse'},
 {'date': '2007/09/28',
  'se

In [65]:
# save the dictionary to json
with open(here/'data_dict.json', 'w') as file:
     file.write(json.dumps(d))

In [66]:
# load the file back in 
with open(here/'data_dict.json', 'r') as file:
     data_dict = json.load(file)

In [67]:
# make sure they are the same
data_dict['00004151'] == d['00004151']

True